In [77]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [57]:
!pip install -q smdebug==0.7.2
!pip install -q sagemaker-experiments==0.1.11

# Specify the S3 Location of the Features

In [58]:
#%store -r scikit_processing_job_s3_output_prefix

In [59]:
scikit_processing_job_s3_output_prefix = 'TT0310826689'

In [60]:
#print(scikit_processing_job_s3_output_prefix)

In [61]:
# HACK:  Remove this once we test the new `05_prepare/` and remove the `is_real_example`

#scikit_processing_job_s3_output_prefix = 'sagemaker-scikit-learn-2020-04-10-04-44-09-647'

print('Previous Scikit Processing Job Name: {}'.format(scikit_processing_job_s3_output_prefix))

Previous Scikit Processing Job Name: TT0310826689


In [62]:
prefix_train = '{}/output/bert-train'.format(scikit_processing_job_s3_output_prefix)
prefix_validation = '{}/output/bert-validation'.format(scikit_processing_job_s3_output_prefix)
prefix_test = '{}/output/bert-test'.format(scikit_processing_job_s3_output_prefix)

train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

# HACK:  TODO

# train_s3_uri = 's3://data-science-on-aws/TT0310826689/bert-train'
# validation_s3_uri = 's3://data-science-on-aws/TT0310826689/bert-validation'
# test_s3_uri = 's3://data-science-on-aws/TT0310826689/bert-test'


In [63]:
# !aws s3 cp --recursive s3://data-science-on-aws/TT0310826689/bert-train/ s3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-train/
# !aws s3 cp --recursive s3://data-science-on-aws/TT0310826689/bert-validation/ s3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-validation/
# !aws s3 cp --recursive s3://data-science-on-aws/TT0310826689/bert-test/ s3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-test/
        

In [64]:
print(train_s3_uri)
!aws s3 ls $train_s3_uri/

s3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-train
2020-04-13 02:48:52 2842017956 part-algo-1-amazon_reviews_us_Apparel_v1_00.tfrecord
2020-04-13 02:48:52 3256644636 part-algo-1-amazon_reviews_us_Books_v1_01.tfrecord
2020-04-13 02:48:52  817388348 part-algo-1-amazon_reviews_us_Digital_Music_Purchase_v1_00.tfrecord
2020-04-13 02:48:52  392412003 part-algo-1-amazon_reviews_us_Furniture_v1_00.tfrecord
2020-04-13 02:48:52 1293434433 part-algo-1-amazon_reviews_us_Home_Improvement_v1_00.tfrecord
2020-04-13 02:49:01  171699748 part-algo-1-amazon_reviews_us_Luggage_v1_00.tfrecord
2020-04-13 02:49:05  450276402 part-algo-1-amazon_reviews_us_Musical_Instruments_v1_00.tfrecord
2020-04-13 02:49:10 1313804795 part-algo-1-amazon_reviews_us_Pet_Products_v1_00.tfrecord
2020-04-13 02:49:15 2379851198 part-algo-1-amazon_reviews_us_Toys_v1_00.tfrecord
2020-04-13 02:49:21 4395944875 part-algo-1-amazon_reviews_us_Wireless_v1_00.tfrecord
2020-04-13 02:49:47 1702370740 part-algo-2-amazon_re

In [65]:
s3_input_train_data = sagemaker.s3_input(s3_data=train_s3_uri, distribution='ShardedByS3Key') 
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_s3_uri, distribution='ShardedByS3Key')
s3_input_test_data = sagemaker.s3_input(s3_data=test_s3_uri, distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-828802286385/TT0310826689/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


In [66]:
!cat src/NEW_tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==0.7.2'])
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig

CLASSES = [1, 2, 3, 4, 5]

def select_data_and_label_from_record(record):
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids']
    }

    y = record['label

# Setup Debugger Rules

In [67]:
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs
from sagemaker.debugger import CollectionConfig
 

rules=[
        Rule.sagemaker(
            rule_configs.loss_not_decreasing(),
            rule_parameters={
                'collection_names': 'losses,metrics',
                'use_losses_collection': 'true',
                'num_steps': '5',
                'diff_percent': '5'
            },
            collections_to_save=[
                CollectionConfig(name='losses',
                                 parameters={
                                     'save_interval': '100',
                                 }),
                CollectionConfig(name='metrics',
                                 parameters={
                                     'save_interval': '100',
                                 })
            ]
        ),
        Rule.sagemaker(
            rule_configs.overtraining(),
            rule_parameters={
                'collection_names': 'losses,metrics',
                'patience_train': '2',
                'patience_validation': '2',
                'delta': '0.5'
            },
            collections_to_save=[
                CollectionConfig(name='losses',
                                 parameters={
                                     'save_interval': '100',
                                 }),
                CollectionConfig(name='metrics',
                                 parameters={
                                     'save_interval': '100',
                                 })
            ]
        )
    ]

# Setup Hyper-Parameters

In [68]:
epochs=1
learning_rate=0.00001 # tune from 1e-5 to 5e-5, step by 1e-5
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=100
validation_steps=100
test_steps=100
train_instance_count=1
train_instance_type='ml.p3.2xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
max_seq_length=128
freeze_bert_layer=False
enable_sagemaker_debugger=False                    # Enable SM Debugger
#use_parameter_server=(train_instance_count >= 2) # Use Parameter Server if distributed cluster
input_mode='File'                                 # 'File' or 'Pipe' Mode
run_validation=True
run_test=True
run_sample_predictions=True

In [69]:
# Track Experiments
import time
unique_id = '{}-{}'.format(input_mode, int(time.time()))

from smexperiments.experiment import Experiment
experiment=Experiment.create(
    experiment_name='train-reviews-bert-{}'.format(unique_id),
    description='Train Reviews BERT', 
    sagemaker_boto_client=sm)

In [70]:
from smexperiments.tracker import Tracker
tracker_display_name='Train-Reviews-BERT-Tracker-{}'.format(unique_id)
print(tracker_display_name)

tracker = Tracker.create(display_name=tracker_display_name, sagemaker_boto_client=sm)
tracker.log_parameters({
    'epochs': epochs,
    'learning_rate': learning_rate,
    'epsilon': epsilon,
    'train_batch_size': train_batch_size,
    'validation_batch_size': validation_batch_size,
    'test_batch_size': test_batch_size,
    'train_steps_per_epoch': train_steps_per_epoch,
    'validation_steps': validation_steps,
    'test_steps': test_steps,
    'train_instance_count': train_instance_count,
    'train_instance_type': train_instance_type,
    'train_volume_size': train_volume_size,
    'use_xla': use_xla,
    'use_amp': use_amp,
    'max_seq_length': max_seq_length,
    'freeze_bert_layer': freeze_bert_layer,
    'enable_sagemaker_debugger': enable_sagemaker_debugger,
#    'use_parameter_server': use_parameter_server,
    'input_mode': input_mode, # 'File' or 'Pipe'
    'run_validation': run_validation,
    'run_test': run_test,
    'run_sample_predictions': run_sample_predictions,    
})
# we can log the s3 uri to the dataset we just uploaded
tracker.log_input(name='reviews_dataset_train', media_type='s3/uri', value=train_s3_uri)
tracker.log_input(name='reviews_dataset_validation', media_type='s3/uri', value=validation_s3_uri)
tracker.log_input(name='reviews_dataset_test', media_type='s3/uri', value=test_s3_uri)

Train-Reviews-BERT-Tracker-File-1586900136


In [71]:
from smexperiments.trial import Trial
trial_name='train-reviews-bert-training-job-{}'.format(unique_id)
trial = Trial.create(trial_name=trial_name, experiment_name=experiment.experiment_name, sagemaker_boto_client=sm)
trial.add_trial_component(tracker.trial_component)
trial_component_display_name='Train-Reviews-BERT-Trial-{}'.format(unique_id)
experiment_config={'ExperimentName': experiment.experiment_name,
                   'TrialName': trial.trial_name,
                   'TrialComponentDisplayName': trial_component_display_name}

In [72]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import DebuggerHookConfig

metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

hook_config = DebuggerHookConfig(
    hook_parameters={
        "save_interval": "100", # number of steps
        "export_tensorboard": "true",
    })

estimator = TensorFlow(entry_point='NEW_tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
                       train_instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       train_instance_type=train_instance_type,
                       train_volume_size=train_volume_size,
                       py_version='py3',
                       framework_version='2.1.0',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train-batch-size': train_batch_size,
                                        'validation-batch-size': validation_batch_size,
                                        'test-batch-size': test_batch_size,                                             
                                        'train-steps-per-epoch': train_steps_per_epoch,
                                        'validation-steps': validation_steps,
                                        'test-steps': test_steps,
                                        'use-xla': use_xla,
                                        'use-amp': use_amp,                                             
                                        'max-seq-length': max_seq_length,
                                        'freeze-bert-layer': freeze_bert_layer,
                                        'enable-sagemaker-debugger': enable_sagemaker_debugger,
                                        'run-validation': run_validation,
                                        'run-test': run_test,
                                        'run-sample-predictions': run_sample_predictions},
#                       distributions={'parameter_server': {'enabled': use_parameter_server}},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
#                       rules=rules,
#                       debugger_hook_config=hook_config,                       
                       train_max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

# Train the Model

In [51]:
estimator.fit(inputs={'train': s3_input_train_data, 
                      'validation': s3_input_validation_data,
                      'test': s3_input_test_data
                     },
                     experiment_config=experiment_config,                   
                     wait=False)

INFO:sagemaker:Creating training-job with name: tensorflow-training-2020-04-14-21-32-07-606


In [52]:
training_job_name = estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  tensorflow-training-2020-04-14-21-32-07-606


In [53]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(region, training_job_name)))

In [54]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, training_job_name)))

In [55]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, training_job_name, region)))

# Track Experiment

In [23]:
from sagemaker.analytics import ExperimentAnalytics

# # Might need to convert ' => "
# search_expression = {
#     'Filters':[
#         {
#             'Name': 'DisplayName',
#             'Operator': 'Equals',
#             'Value': 'Training'
#         }
#     ]
# }

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=experiment.experiment_name,
#    search_expression=search_expression,
    sort_by='metrics.validation:accuracy.max',
    sort_order='Descending',
    metric_names=['validation:accuracy'],
    parameter_names=['epochs', 'train_batch_size']
)

In [24]:
analytics_table = trial_component_analytics.dataframe()
analytics_table

,TrialComponentName,DisplayName,SourceArn,epochs
0,tensorflow-training-2020-04-14-19-27-45-657-aw...,Train-Reviews-BERT-Trial-File-1586892463,arn:aws:sagemaker:us-east-1:828802286385:train...,1.0
1,TrialComponent-2020-04-14-192743-vlcr,Train-Reviews-BERT-Tracker-File-1586892463,NaN,NaN


In [25]:
# search_expression={
#     'Filters':[{
#         'Name': 'Parents.TrialName',
#         'Operator': 'Equals',
#         'Value': ??
#     }]
# },

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment.experiment_name,
#    search_expression=search_expression,
    sort_by="CreationTime",
    sort_order="Ascending",
)
lineage_table.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,enable-sagemaker-debugger,epochs,epsilon,...,sagemaker_region,sagemaker_submit_directory,test-batch-size,test-steps,train-batch-size,train-steps-per-epoch,use-amp,use-xla,validation-batch-size,validation-steps
0,TrialComponent-2020-04-14-192743-vlcr,Train-Reviews-BERT-Tracker-File-1586892463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tensorflow-training-2020-04-14-19-27-45-657-aw...,Train-Reviews-BERT-Trial-File-1586892463,arn:aws:sagemaker:us-east-1:828802286385:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/t...,1.0,ml.p3.8xlarge,1024.0,true,1.0,1.000000e-08,...,"""us-east-1""","""s3://sagemaker-us-east-1-828802286385/tensorf...",128.0,100.0,128.0,100.0,true,true,128.0,100.0


# Retrieving and Analyzing Debugger Results

## Analyze Rule Status

In [26]:
estimator.latest_training_job.rule_job_summary()

[{'RuleConfigurationName': 'LossNotDecreasing',
  'RuleEvaluationStatus': 'InProgress',
  'LastModifiedTime': datetime.datetime(2020, 4, 14, 19, 27, 48, 68000, tzinfo=tzlocal())},
 {'RuleConfigurationName': 'Overtraining',
  'RuleEvaluationStatus': 'InProgress',
  'LastModifiedTime': datetime.datetime(2020, 4, 14, 19, 27, 48, 68000, tzinfo=tzlocal())}]

## Analyze Tensors

Before getting to analysis, here are some notes on concepts being used in Debugger that help with analysis.
* **Trial** - object that is a center piece of Debugger API when it comes to getting access to tensors. It is a top level abstract that represents a single run of a training job. All tensors emitted by training job are associated with its trial.
* **Step** - object that represents next level of abstraction. In Debugger - step is a representation of a single batch of a training job. Each trial has multiple steps. Each tensor is associated with multiple steps - having a particular value at each of the steps.
* **Tensor** - object that represent actual tensor saved during training job. Note, a tensor can be a 1-D scaler, as well (ie. loss is stored as a scalar).

For more details on these concepts as well as on Debugger API in general (including examples) please refer to Debugger Analysis API documentation.

In [27]:
from smdebug.trials import create_trial

# this is where we create a Trial object that allows access to saved tensors
trial = create_trial(estimator.latest_job_debugger_artifacts_path())

[2020-04-14 19:27:57.857 ip-172-16-9-13:31306 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-828802286385/tensorflow-training-2020-04-14-19-27-45-657/debug-output


KeyboardInterrupt: 

In [ ]:
for i in trial.tensor_names():
    print(i)

In [ ]:
import pandas as pd

def get_data(trial, tensor_name, batch_index, steps_range, mode):
    tensor = trial.tensor(tensor_name)
    vals = []
    for step_num in steps_range:
        val = tensor.value(step_num=step_num, mode=mode)[batch_index]
        vals.append(val)
    return pd.DataFrame(columns=['steps', tensor_name], data=list(zip(steps_range, vals)))


In [ ]:
from smdebug.tensorflow import modes

# Below we select the very first tensor from every batch.
# Feel free to modify this and select another tensor from the batch.
batch_index = 0

# This is a name of a tensor to analyze.
tensor_name = 'accuracy'

steps = 0
while steps == 0:
    # trial.steps return all steps that have been downloaded by Debugger to date.
    # It doesn't represent all steps that are to be available once training job is complete -
    # it is a snapshot of a current state of the training job. If you call it after training job is done
    # you will get all tensors available at once.
    steps = trial.steps()
    print('Waiting for tensors to become available...')
    time.sleep(3)
print('\nDone')

print('Getting tensors and plotting...')
rendered_steps = []

# trial.loaded_all_steps is a way to keep monitoring for a state of a training job as seen by Debugger.
# When SageMaker completes training job Debugger, and trial, becomes aware of it.

loaded_all_steps = False
while not loaded_all_steps:
    loaded_all_steps = trial.loaded_all_steps
    steps = trial.steps()
    # show diff between lists
    steps_to_render = list(set(steps).symmetric_difference(set(rendered_steps)))
    
    data = get_data(trial=trial, 
                    tensor_name=tensor_name, 
                    batch_index=0, 
                    steps_range=steps_to_render, 
                    mode=modes.GLOBAL)
    print(data)
    data.plot(x='steps', y=tensor_name)
    
    rendered_steps.extend(steps_to_render)
    time.sleep(5)

## Visualize Loss

In [ ]:
from smdebug.tensorflow import modes

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

#from matplotlib import pyplot as plt

import pandas as pd

# Let's visualize weights of the first convolutional layer as they progressively change through training.
tensor_name = 'loss'

num_batches = trial.tensor(tensor_name).value(step_num=steps[0]).shape[0]
for batch_index in range(0, num_batches):
    steps_range = trial.tensor(tensor_name).steps()
    print(steps_range)
    data = get_data(trial=trial, 
                    tensor_name=tensor_name, 
                    batch_index=batch_index, 
                    steps_range=steps_range, 
                    mode=modes.GLOBAL)
    print(data)
    data.plot(x='steps', y=tensor_name)

## Visualize Accuracy

In [ ]:
from smdebug.tensorflow import modes

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

#from matplotlib import pyplot as plt

import pandas as pd

tensor_name = 'accuracy'

num_batches = trial.tensor(tensor_name).value(step_num=steps[0]).shape[0]
for batch_index in range(0, num_batches):
    steps_range = trial.tensor(tensor_name).steps()
    print(steps_range)
    data = get_data(trial=trial, 
                    tensor_name=tensor_name, 
                    batch_index=batch_index, 
                    steps_range=steps_range, 
                    mode=modes.GLOBAL)
    print(data)
    data.plot(x='steps', y=tensor_name)

## Add Lambda Function to Stop the TrainingJob Early
In your AWS console, go to Lambda Management Console,
Create a new function by hitting Create Function,
Choose the language as Python 3.7 and put in the following sample code for stopping the training job if one of the Rule statuses is "IssuesFound".

### CloudWatch Events for Rules
Rule status changes in a training job trigger CloudWatch Events. These events can be acted upon by configuring a CloudWatch Rule (different from Amazon SageMaker Debugger Rule) to trigger each time a Debugger Rule changes status. In this notebook we'll go through how you can create a CloudWatch Rule to direct Training Job State change events to a lambda function that stops the training job in case a rule triggers and has status "IssuesFound".

Create a new execution role for the Lambda, and
In your IAM console, search for the role and attach "AmazonSageMakerFullAccess" policy to the role. This is needed for the code in your Lambda function to stop the training job.


### Create a CloudWatch Rule to Trigger a Lamba
In your AWS Console, go to CloudWatch and select Rule from the left column,
Hit Create Rule. The console will redirect you to the Rule creation page,
For the Service Name, select "SageMaker".
For the Event Type, select "SageMaker Training Job State Change".
In the Targets select the Lambda function you created above, and
For this example notebook, we'll leave everything as is.


#### Create the Lambda

```
import json
import boto3
import logging

def lambda_handler(event, context):
    training_job_name = event.get("detail").get("TrainingJobName")
    eval_statuses = event.get("detail").get("DebugRuleEvaluationStatuses", None)

    if eval_statuses is None or len(eval_statuses) == 0:
        logging.info("Couldn't find any debug rule statuses, skipping...")
        return {
            'statusCode': 200,
            'body': json.dumps('Nothing to do')
        }

    client = boto3.client('sagemaker')

    for status in eval_statuses:
        if status.get("RuleEvaluationStatus") == "IssuesFound":
            logging.info(
                'Evaluation of rule configuration {} resulted in "IssuesFound". '
                'Attempting to stop training job {}'.format(
                    status.get("RuleConfigurationName"), training_job_name
                )
            )
            try:
                client.stop_training_job(
                    TrainingJobName=training_job_name
                )
            except Exception as e:
                logging.error(
                    "Encountered error while trying to "
                    "stop training job {}: {}".format(
                        training_job_name, str(e)
                    )
                )
                raise e
    return None
```

* Create a new execution role for the Lambda, and
* In your IAM console, search for the role and attach "AmazonSageMakerFullAccess" policy to the role. This is needed for the code in your Lambda function to stop the training job.

#### Create a CloudWatch Rule

* In your AWS Console, go to CloudWatch and select Rule from the left column,
* Hit Create Rule. The console will redirect you to the Rule creation page,
 * For the Service Name, select "SageMaker".
 * For the Event Type, select "SageMaker Training Job State Change".
* In the Targets select the Lambda function you created above, and
* For this example notebook, we'll leave everything as is.

SageMaker kicked off rule evaluation jobs, one for each of the SageMaker rules - `Overtraining` and `LossNotDecreasing` as specified in the estimator. If we setup a CloudWatch Rule to stop the training job, we would see the `TrainingJobStatus` change to `Stopped` once the `RuleEvaluationStatus` for changes to `IssuesFound`.

In [ ]:
# # This utility gives the link to monitor the CW event
# def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
#         """Helper function to get the rule job name"""
#         return "{}-{}-{}".format(
#             training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
#         )
    
# def _get_cw_url_for_rule_job(rule_job_name, region):
#     return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(region, region, rule_job_name)


# def get_rule_jobs_cw_urls(estimator):
#     region = boto3.Session().region_name
#     training_job = estimator.latest_training_job
#     training_job_name = training_job.describe()["TrainingJobName"]
#     rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]
    
#     result={}
#     for status in rule_eval_statuses:
#         if status.get("RuleEvaluationJobArn", None) is not None:
#             rule_job_name = _get_rule_job_name(training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"])
#             result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(rule_job_name, region)
#     return result

# get_rule_jobs_cw_urls(estimator)

In [ ]:
# estimator.latest_training_job.describe()["TrainingJobStatus"]


# Setup Hyper-Parameter Ranges to Explore


In [78]:
# Note:  enable_sagemaker_debugger must be disabled or these jobs will fail 
#        on the call to `config_from_jsonfile()`

from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import CategoricalParameter
from sagemaker.tuner import HyperparameterTuner
                                                
hyperparameter_ranges = {
#    'use-xla': CategoricalParameter([True, False]),
#    'use-amp': CategoricalParameter([True, False]),
    'epochs': IntegerParameter(2, 16, scaling_type='Logarithmic'),
    'learning_rate': ContinuousParameter(0.00001, 0.00005, scaling_type='Linear'),
#    'train-steps-per-epoch': IntegerParameter(10, 1000, scaling_type='Logarithmic'),    
    'train-batch-size': CategoricalParameter([128, 256, 512]),
    'freeze-bert-layer': CategoricalParameter([True, False])
}

objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=6,
    max_parallel_jobs=2,
    strategy='Bayesian',
    early_stopping_type='Auto'
)

# Start Tuning Job

In [79]:
tuner.fit({'train': s3_input_train_data, 
           'validation': s3_input_validation_data,
           'test': s3_input_test_data
          }, include_cls_metadata=False)

INFO:root:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: tensorflow-training-200414-2142


##  Check Tuning Job Status
Re-run this cell to track the status

In [80]:
from pprint import pprint

tuning_job_name = tuner.latest_tuning_job.job_name

job_description = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = job_description['HyperParameterTuningJobStatus']

print('\n')
print(status)
print('\n')
pprint(job_description)

if status != 'Completed':
    job_count = job_description['TrainingJobStatusCounters']['Completed']
    print('Not yet complete, but {} jobs have completed.')
    
    if job_description.get('BestTrainingJob', None):
        print("Best candidate:")
        pprint(job_description['BestTrainingJob']['TrainingJobName'])
        pprint(job_description['BestTrainingJob']['FinalHyperParameterTuningJobObjectiveMetric'])
    else:
        print("No training jobs have reported results yet.")    



InProgress


{'CreationTime': datetime.datetime(2020, 4, 14, 21, 42, 35, 39000, tzinfo=tzlocal()),
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:828802286385:hyper-parameter-tuning-job/tensorflow-training-200414-2142',
 'HyperParameterTuningJobConfig': {'HyperParameterTuningJobObjective': {'MetricName': 'validation:accuracy',
                                                                        'Type': 'Maximize'},
                                   'ParameterRanges': {'CategoricalParameterRanges': [{'Name': 'train-batch-size',
                                                                                       'Values': ['"128"',
                                                                                                  '"256"',
                                                                                                  '"512"']},
                                                                                      {'Name': 'freeze-bert-layer',
         

## Show the Tuning Jobs
Note:  This will fail at first.  Please wait about 30-60 seconds and re-run.

In [84]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

hp_results = HyperparameterTuningJobAnalytics(
    sagemaker_session=sess, 
    hyperparameter_tuning_job_name=tuning_job_name
)

df_results = hp_results.dataframe()

df_results.sort_values('FinalObjectiveValue', ascending=0)

,FinalObjectiveValue,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,epochs,freeze-bert-layer,learning_rate,train-batch-size
0,None,None,tensorflow-training-200414-2142-002-1a86aaea,InProgress,None,6.0,"""False""",0.000011,"""128"""
1,None,None,tensorflow-training-200414-2142-001-cc9acb6a,InProgress,None,3.0,"""True""",0.000037,"""128"""


## Show the Best Candidate

In [42]:
df_results.sort_values('FinalObjectiveValue', ascending=0).head(1)

,FinalObjectiveValue,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,epochs,freeze-bert-layer,learning_rate,train-batch-size,train-steps-per-epoch,use-amp,use-xla
0,None,None,tensorflow-training-200414-1929-002-8516e6d0,InProgress,None,16.0,"""False""",0.000047,"""256""",16.0,"""False""","""True"""


In [43]:
from IPython.core.display import display, HTML
    
display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/{}">Hyper-Parameter Tuning Job</a></b>'.format(region, tuning_job_name)))

# Download and Load the Trained Model

In [ ]:
models_dir = './models'

In [ ]:
# Download the model and output artifacts from AWS S3
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

In [ ]:
import tarfile
import pickle as pkl

#!ls -al ./models

tar = tarfile.open('{}/model.tar.gz'.format(models_dir))
tar.extractall(path=models_dir)
tar.close()

In [ ]:
!ls -al $models_dir

In [ ]:
# Install TensorFlow and Transformers

In [ ]:
!pip install -q pip --upgrade
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0 --upgrade --ignore-installed
!pip install -q transformers==2.8.0

In [ ]:
!pip list

# Load the model

In [ ]:
transformer_model_dir = '{}/transformer/pretrained/'.format(models_dir)

!ls -al $transformer_model_dir

In [ ]:
cat $transformer_model_dir/config.json

In [ ]:
import os
from transformers import TFDistilBertForSequenceClassification

loaded_model = TFDistilBertForSequenceClassification.from_pretrained(transformer_model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     })

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inference_device = -1 # CPU: -1, GPU: 0
print('inference_device {}'.format(inference_device))

In [ ]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=loaded_model, 
                                                tokenizer=tokenizer,
                                                framework='tf',
                                                device=inference_device) # -1 is CPU, >= 0 is GPU

In [ ]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

In [ ]:
review = """Really bad.  I hope they don't make this anymore."""
print(review, inference_pipeline(review))

# Uncomment to Start Tensorboard
Let's start Tensorboard and point to the tensorboard logs that we downloaded from S3 directly.

Note:  If you pointed Tensorboard to S3 directly, you must prepend this command with `S3_REGION=[your-region]`.

In [ ]:
outputs_dir = './outputs'

In [ ]:
!aws s3 cp --recursive s3://$bucket/$training_job_name/debug-output/events/ $outputs_dir/events/

In [ ]:
!ls -al $outputs_dir/events

In [ ]:
#!S3_REGION=us-east-1 tensorboard --port 6006 --logdir s3://$bucket/$training_job_name/debug-output/events/  # <== MAKE SURE YOU INCLUDE THE TRAILING `/`
!S3_REGION=us-east-1 tensorboard --port 6006 --logdir $outputs_dir/events/ # <== MAKE SURE YOU INCLUDE THE TRAILING `/`

While Tensorboard is running locally on your SageMaker Notebook instance, it is reading the training logs from Amazon S3.

Navigate to https://workshop.notebook.[your-region].sagemaker.aws/proxy/6006/  <== MAKE SURE YOU INCLUDE THE TRAILING SLASH

_Note:  Make sure you copy the trailing `/` in the link above.  If you see no data, you are likely not using the correct S3 bucket above._

![Tensorboard](img/tensorboard.png)

# Stop Tensorboard
Once you are done, hit `Kernel => Stop` to stop the running `Tensorboard` process in this notebook.